In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
df = pd.read_csv("../data/train/variants/fr/variants_hist_data.csv", sep =';')
df2 = pd.read_csv("../data/train/all_data_merged/fr/Enriched_Covid_history_data.csv")
df2["time"] = pd.to_datetime(df2["time"])
df['dep'] = df['dep'].replace({'2A':'201','2B':'202'}).astype(int)
df = df[df['dep']<203]
#df = df.groupby(["dep","semaine"], level = -1).sum()["dep","semaine","Nb_susp_501Y_V1","Nb_susp_501Y_V2_3"]
df= df.groupby(['dep', 'semaine'])[["dep","semaine","Nb_susp_501Y_V1","Nb_susp_501Y_V2_3"]].sum().drop(columns = ["dep"]).reset_index()
df

,dep,semaine,Nb_susp_501Y_V1,Nb_susp_501Y_V2_3
0,1,2021-02-12-2021-02-18,686,42
1,1,2021-02-13-2021-02-19,706,46
2,1,2021-02-14-2021-02-20,730,46
3,1,2021-02-15-2021-02-21,748,48
4,1,2021-02-16-2021-02-22,852,52
...,...,...,...,...
4219,202,2021-03-23-2021-03-29,302,14
4220,202,2021-03-24-2021-03-30,302,18
4221,202,2021-03-25-2021-03-31,328,18
4222,202,2021-03-26-2021-04-01,318,16


In [3]:
#10 premiers caractères datetime
#réaffecter directement
#récupérer la date minimum de variant_list (=df avec datetime)

def to_datalist(row):
    date = pd.to_datetime(row["semaine"][0:10], yearfirst = True)
    return date

df['jour'] = df.apply(to_datalist, axis=1)
df.drop(columns='semaine', inplace=True)
df.rename(columns={'jour':'semaine'}, inplace=True)

In [4]:
def create_possibilities(row):
    return pd.date_range(start = row['semaine'], periods=7).tolist()

df['7_days'] = df.apply(create_possibilities, axis=1)
df

,dep,Nb_susp_501Y_V1,Nb_susp_501Y_V2_3,semaine,7_days
0,1,686,42,2021-02-12,"[2021-02-12 00:00:00, 2021-02-13 00:00:00, 202..."
1,1,706,46,2021-02-13,"[2021-02-13 00:00:00, 2021-02-14 00:00:00, 202..."
2,1,730,46,2021-02-14,"[2021-02-14 00:00:00, 2021-02-15 00:00:00, 202..."
3,1,748,48,2021-02-15,"[2021-02-15 00:00:00, 2021-02-16 00:00:00, 202..."
4,1,852,52,2021-02-16,"[2021-02-16 00:00:00, 2021-02-17 00:00:00, 202..."
...,...,...,...,...,...
4219,202,302,14,2021-03-23,"[2021-03-23 00:00:00, 2021-03-24 00:00:00, 202..."
4220,202,302,18,2021-03-24,"[2021-03-24 00:00:00, 2021-03-25 00:00:00, 202..."
4221,202,328,18,2021-03-25,"[2021-03-25 00:00:00, 2021-03-26 00:00:00, 202..."
4222,202,318,16,2021-03-26,"[2021-03-26 00:00:00, 2021-03-27 00:00:00, 202..."


In [5]:
referencedate = df['semaine'].min()

In [ ]:
# regarder pour chaque row de Enriched si :
# - le jour est avant le jour de début des variants
# - le jour est dans la semaine suivant le row des variants
# - append à Enriched+variant (= df de enriched + infos sur les variants)

In [45]:
def check_variant(v_row, date):
    if date in v_row['7_days']:
        print(date, v_row['Nb_susp_501Y_V1'], v_row['Nb_susp_501Y_V2_3'] )
#        print(v_row['Nb_susp_501Y_V1'], v_row['Nb_susp_501Y_V2_3'])
        return v_row['Nb_susp_501Y_V1'], v_row['Nb_susp_501Y_V2_3']
            
    

In [65]:
def enriched_variant(row):
    date = row['time']
    depnum = row['numero']
    if date < referencedate:
        row['Nb_susp_501Y_V1'], row['Nb_susp_501Y_V2_3'] = 0, 0
    else : 
        df_dep = df[df['dep']==depnum]
        ans=[]
        res = df_dep.apply(check_variant,date=date, axis=1)
        
        ans.append(next((el for el in res if el is not None), None))
        print(ans)
        row['Nb_susp_501Y_V1'], row['Nb_susp_501Y_V2_3'] = ans[0][0], ans[0][1] 
    return None

In [66]:
df2.apply(enriched_variant, axis=1)

2021-02-12 00:00:00 686 42
[(686, 42)]
2021-02-13 00:00:00 686 42
2021-02-13 00:00:00 706 46
[(686, 42)]
2021-02-14 00:00:00 686 42
2021-02-14 00:00:00 706 46
2021-02-14 00:00:00 730 46
[(686, 42)]
2021-02-15 00:00:00 686 42
2021-02-15 00:00:00 706 46
2021-02-15 00:00:00 730 46
2021-02-15 00:00:00 748 48
[(686, 42)]
2021-02-16 00:00:00 686 42
2021-02-16 00:00:00 706 46
2021-02-16 00:00:00 730 46
2021-02-16 00:00:00 748 48
2021-02-16 00:00:00 852 52
[(686, 42)]
2021-02-17 00:00:00 686 42
2021-02-17 00:00:00 706 46
2021-02-17 00:00:00 730 46
2021-02-17 00:00:00 748 48
2021-02-17 00:00:00 852 52
2021-02-17 00:00:00 884 72
[(686, 42)]
2021-02-18 00:00:00 686 42
2021-02-18 00:00:00 706 46
2021-02-18 00:00:00 730 46
2021-02-18 00:00:00 748 48
2021-02-18 00:00:00 852 52
2021-02-18 00:00:00 884 72
2021-02-18 00:00:00 922 78
[(686, 42)]
2021-02-19 00:00:00 706 46
2021-02-19 00:00:00 730 46
2021-02-19 00:00:00 748 48
2021-02-19 00:00:00 852 52
2021-02-19 00:00:00 884 72
2021-02-19 00:00:00 922 7

2021-02-13 00:00:00 104 6
2021-02-13 00:00:00 132 4
[(104, 6)]
2021-02-14 00:00:00 104 6
2021-02-14 00:00:00 132 4
2021-02-14 00:00:00 142 4
[(104, 6)]
2021-02-15 00:00:00 104 6
2021-02-15 00:00:00 132 4
2021-02-15 00:00:00 142 4
2021-02-15 00:00:00 142 4
[(104, 6)]
2021-02-16 00:00:00 104 6
2021-02-16 00:00:00 132 4
2021-02-16 00:00:00 142 4
2021-02-16 00:00:00 142 4
2021-02-16 00:00:00 156 4
[(104, 6)]
2021-02-17 00:00:00 104 6
2021-02-17 00:00:00 132 4
2021-02-17 00:00:00 142 4
2021-02-17 00:00:00 142 4
2021-02-17 00:00:00 156 4
2021-02-17 00:00:00 154 2
[(104, 6)]
2021-02-18 00:00:00 104 6
2021-02-18 00:00:00 132 4
2021-02-18 00:00:00 142 4
2021-02-18 00:00:00 142 4
2021-02-18 00:00:00 156 4
2021-02-18 00:00:00 154 2
2021-02-18 00:00:00 152 4
[(104, 6)]
2021-02-19 00:00:00 132 4
2021-02-19 00:00:00 142 4
2021-02-19 00:00:00 142 4
2021-02-19 00:00:00 156 4
2021-02-19 00:00:00 154 2
2021-02-19 00:00:00 152 4
2021-02-19 00:00:00 170 4
[(132, 4)]
2021-02-20 00:00:00 142 4
2021-02-20 00

2021-02-23 00:00:00 304 8
2021-02-23 00:00:00 330 8
2021-02-23 00:00:00 364 8
2021-02-23 00:00:00 358 8
2021-02-23 00:00:00 362 8
2021-02-23 00:00:00 360 8
2021-02-23 00:00:00 354 8
[(304, 8)]
2021-02-24 00:00:00 330 8
2021-02-24 00:00:00 364 8
2021-02-24 00:00:00 358 8
2021-02-24 00:00:00 362 8
2021-02-24 00:00:00 360 8
2021-02-24 00:00:00 354 8
2021-02-24 00:00:00 330 4
[(330, 8)]
2021-02-25 00:00:00 364 8
2021-02-25 00:00:00 358 8
2021-02-25 00:00:00 362 8
2021-02-25 00:00:00 360 8
2021-02-25 00:00:00 354 8
2021-02-25 00:00:00 330 4
2021-02-25 00:00:00 314 4
[(364, 8)]
2021-02-26 00:00:00 358 8
2021-02-26 00:00:00 362 8
2021-02-26 00:00:00 360 8
2021-02-26 00:00:00 354 8
2021-02-26 00:00:00 330 4
2021-02-26 00:00:00 314 4
2021-02-26 00:00:00 286 4
[(358, 8)]
2021-02-27 00:00:00 362 8
2021-02-27 00:00:00 360 8
2021-02-27 00:00:00 354 8
2021-02-27 00:00:00 330 4
2021-02-27 00:00:00 314 4
2021-02-27 00:00:00 286 4
2021-02-27 00:00:00 292 6
[(362, 8)]
2021-02-28 00:00:00 360 8
2021-02-2

2021-02-24 00:00:00 94 12
2021-02-24 00:00:00 80 10
2021-02-24 00:00:00 90 8
[(102, 16)]
2021-02-25 00:00:00 102 14
2021-02-25 00:00:00 96 10
2021-02-25 00:00:00 94 12
2021-02-25 00:00:00 94 12
2021-02-25 00:00:00 80 10
2021-02-25 00:00:00 90 8
2021-02-25 00:00:00 104 2
[(102, 14)]
2021-02-26 00:00:00 96 10
2021-02-26 00:00:00 94 12
2021-02-26 00:00:00 94 12
2021-02-26 00:00:00 80 10
2021-02-26 00:00:00 90 8
2021-02-26 00:00:00 104 2
2021-02-26 00:00:00 110 2
[(96, 10)]
2021-02-27 00:00:00 94 12
2021-02-27 00:00:00 94 12
2021-02-27 00:00:00 80 10
2021-02-27 00:00:00 90 8
2021-02-27 00:00:00 104 2
2021-02-27 00:00:00 110 2
2021-02-27 00:00:00 128 4
[(94, 12)]
2021-02-28 00:00:00 94 12
2021-02-28 00:00:00 80 10
2021-02-28 00:00:00 90 8
2021-02-28 00:00:00 104 2
2021-02-28 00:00:00 110 2
2021-02-28 00:00:00 128 4
2021-02-28 00:00:00 144 2
[(94, 12)]
2021-03-01 00:00:00 80 10
2021-03-01 00:00:00 90 8
2021-03-01 00:00:00 104 2
2021-03-01 00:00:00 110 2
2021-03-01 00:00:00 128 4
2021-03-01 0

2021-02-23 00:00:00 640 22
2021-02-23 00:00:00 694 22
2021-02-23 00:00:00 766 22
2021-02-23 00:00:00 748 28
2021-02-23 00:00:00 714 28
2021-02-23 00:00:00 714 28
2021-02-23 00:00:00 810 20
[(640, 22)]
2021-02-24 00:00:00 694 22
2021-02-24 00:00:00 766 22
2021-02-24 00:00:00 748 28
2021-02-24 00:00:00 714 28
2021-02-24 00:00:00 714 28
2021-02-24 00:00:00 810 20
2021-02-24 00:00:00 792 24
[(694, 22)]
2021-02-25 00:00:00 766 22
2021-02-25 00:00:00 748 28
2021-02-25 00:00:00 714 28
2021-02-25 00:00:00 714 28
2021-02-25 00:00:00 810 20
2021-02-25 00:00:00 792 24
2021-02-25 00:00:00 780 32
[(766, 22)]
2021-02-26 00:00:00 748 28
2021-02-26 00:00:00 714 28
2021-02-26 00:00:00 714 28
2021-02-26 00:00:00 810 20
2021-02-26 00:00:00 792 24
2021-02-26 00:00:00 780 32
2021-02-26 00:00:00 772 30
[(748, 28)]
2021-02-27 00:00:00 714 28
2021-02-27 00:00:00 714 28
2021-02-27 00:00:00 810 20
2021-02-27 00:00:00 792 24
2021-02-27 00:00:00 780 32
2021-02-27 00:00:00 772 30
2021-02-27 00:00:00 804 22
[(714, 

...

2021-02-18 00:00:00 2076 94
2021-02-18 00:00:00 2236 118
2021-02-18 00:00:00 2294 138
2021-02-18 00:00:00 2312 138
2021-02-18 00:00:00 2508 184
2021-02-18 00:00:00 2717 204
2021-02-18 00:00:00 2899 206
[(2076, 94)]
2021-02-19 00:00:00 2236 118
2021-02-19 00:00:00 2294 138
2021-02-19 00:00:00 2312 138
2021-02-19 00:00:00 2508 184
2021-02-19 00:00:00 2717 204
2021-02-19 00:00:00 2899 206
2021-02-19 00:00:00 2916 230
[(2236, 118)]
2021-02-20 00:00:00 2294 138
2021-02-20 00:00:00 2312 138
2021-02-20 00:00:00 2508 184
2021-02-20 00:00:00 2717 204
2021-02-20 00:00:00 2899 206
2021-02-20 00:00:00 2916 230
2021-02-20 00:00:00 3100 232
[(2294, 138)]
2021-02-21 00:00:00 2312 138
2021-02-21 00:00:00 2508 184
2021-02-21 00:00:00 2717 204
2021-02-21 00:00:00 2899 206
2021-02-21 00:00:00 2916 230
2021-02-21 00:00:00 3100 232
2021-02-21 00:00:00 3206 218
[(2312, 138)]
2021-02-22 00:00:00 2508 184
2021-02-22 00:00:00 2717 204
2021-02-22 00:00:00 2899 206
2021-02-22 00:00:00 2916 230
2021-02-22 00:00:0

2021-02-22 00:00:00 4298 514
2021-02-22 00:00:00 4292 514
[(3667, 312)]
2021-02-23 00:00:00 3864 324
2021-02-23 00:00:00 4002 352
2021-02-23 00:00:00 4144 378
2021-02-23 00:00:00 4154 460
2021-02-23 00:00:00 4298 514
2021-02-23 00:00:00 4292 514
2021-02-23 00:00:00 4320 535
[(3864, 324)]
2021-02-24 00:00:00 4002 352
2021-02-24 00:00:00 4144 378
2021-02-24 00:00:00 4154 460
2021-02-24 00:00:00 4298 514
2021-02-24 00:00:00 4292 514
2021-02-24 00:00:00 4320 535
2021-02-24 00:00:00 4332 555
[(4002, 352)]
2021-02-25 00:00:00 4144 378
2021-02-25 00:00:00 4154 460
2021-02-25 00:00:00 4298 514
2021-02-25 00:00:00 4292 514
2021-02-25 00:00:00 4320 535
2021-02-25 00:00:00 4332 555
2021-02-25 00:00:00 4436 565
[(4144, 378)]
2021-02-26 00:00:00 4154 460
2021-02-26 00:00:00 4298 514
2021-02-26 00:00:00 4292 514
2021-02-26 00:00:00 4320 535
2021-02-26 00:00:00 4332 555
2021-02-26 00:00:00 4436 565
2021-02-26 00:00:00 4452 555
[(4154, 460)]
2021-02-27 00:00:00 4298 514
2021-02-27 00:00:00 4292 514
20

2021-03-01 00:00:00 3924 344
2021-03-01 00:00:00 4024 312
2021-03-01 00:00:00 4216 336
2021-03-01 00:00:00 4244 370
2021-03-01 00:00:00 4360 364
2021-03-01 00:00:00 4338 352
2021-03-01 00:00:00 4364 350
[(3924, 344)]
2021-03-02 00:00:00 4024 312
2021-03-02 00:00:00 4216 336
2021-03-02 00:00:00 4244 370
2021-03-02 00:00:00 4360 364
2021-03-02 00:00:00 4338 352
2021-03-02 00:00:00 4364 350
2021-03-02 00:00:00 4507 364
[(4024, 312)]
2021-03-03 00:00:00 4216 336
2021-03-03 00:00:00 4244 370
2021-03-03 00:00:00 4360 364
2021-03-03 00:00:00 4338 352
2021-03-03 00:00:00 4364 350
2021-03-03 00:00:00 4507 364
2021-03-03 00:00:00 4663 366
[(4216, 336)]
2021-03-04 00:00:00 4244 370
2021-03-04 00:00:00 4360 364
2021-03-04 00:00:00 4338 352
2021-03-04 00:00:00 4364 350
2021-03-04 00:00:00 4507 364
2021-03-04 00:00:00 4663 366
2021-03-04 00:00:00 4739 392
[(4244, 370)]
2021-03-05 00:00:00 4360 364
2021-03-05 00:00:00 4338 352
2021-03-05 00:00:00 4364 350
2021-03-05 00:00:00 4507 364
2021-03-05 00:00

2021-03-20 00:00:00 9409 412
[(8253, 460)]
2021-03-21 00:00:00 8229 460
2021-03-21 00:00:00 8541 460
2021-03-21 00:00:00 9014 450
2021-03-21 00:00:00 9302 450
2021-03-21 00:00:00 9178 426
2021-03-21 00:00:00 9409 412
2021-03-21 00:00:00 9561 432
[(8229, 460)]
2021-03-22 00:00:00 8541 460
2021-03-22 00:00:00 9014 450
2021-03-22 00:00:00 9302 450
2021-03-22 00:00:00 9178 426
2021-03-22 00:00:00 9409 412
2021-03-22 00:00:00 9561 432
2021-03-22 00:00:00 9557 428
[(8541, 460)]
2021-03-23 00:00:00 9014 450
2021-03-23 00:00:00 9302 450
2021-03-23 00:00:00 9178 426
2021-03-23 00:00:00 9409 412
2021-03-23 00:00:00 9561 432
2021-03-23 00:00:00 9557 428
2021-03-23 00:00:00 9496 396
[(9014, 450)]
2021-03-24 00:00:00 9302 450
2021-03-24 00:00:00 9178 426
2021-03-24 00:00:00 9409 412
2021-03-24 00:00:00 9561 432
2021-03-24 00:00:00 9557 428
2021-03-24 00:00:00 9496 396
2021-03-24 00:00:00 8643 364
[(9302, 450)]
2021-03-25 00:00:00 9178 426
2021-03-25 00:00:00 9409 412
2021-03-25 00:00:00 9561 432
20

0        None
1        None
2        None
3        None
4        None
         ... 
30585    None
30586    None
30587    None
30588    None
30589    None
Length: 30590, dtype: object

In [67]:
df2[['time', 'Nb_susp_501Y_V1', 'Nb_susp_501Y_V2_3']]

,time,Nb_susp_501Y_V1,Nb_susp_501Y_V2_3
0,2020-05-14,0,0
1,2020-05-15,0,0
2,2020-05-16,0,0
3,2020-05-17,0,0
4,2020-05-18,0,0
...,...,...,...
30585,2021-03-27,9561,432
30586,2021-03-28,9557,428
30587,2021-03-29,9496,396
30588,2021-03-30,8643,364
